# Taller 6

## Métodos Computacionales 1 2022-10

Instrucciones: Suba el jupyter notebook **.ipynb** en la actividad de bloque neon. El archivo debe ser nombrado ApellidoNombre_ApellidoNombre_Taller6.ipynb. **El archivo debe poder correr en el binder de la magistral** 

Si un ejercicio demora **más de 10 segundos** en correr se considerará **incorrecto**.

No modifique el cuaderno, ni los casos de prueba, puede correr los casos de prueba pero no modificarlos.

**Entrega: Lunes 16 de Mayo 11:59 pm**

## Librerias

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal 

## Problema 1

Resolver los siguientes problemas con 30000 pasos de MonteCarlo. Además planteé la ecuación que resuelve el problema

a) Un examen de opción múltiple contiene 30 preguntas, cada una con cinco respuestas posibles. Supongase que el estudiante sólo adivina las respuestas. ¿Cuál es la probabilidad de que el estudiante conteste de manera correcta más de 20 preguntas? 

a) En una central teléfonica se reciben llamadas según las leyes de un proceso de Poisson, con un promedio de diez llamadas por hora, ¿Cuál es la probabilidad de que ninguna llamada sea recibida entre las 8am y las 12 del medio día?

Escriba aquí tus ecuaciones aquí

In [2]:
# tu codigo aqui

## Problema 2

Considere las dos caracteristicas "Alcohol level" y "Alcalinity level" del conjunto de datos [wine_data](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html). Pueden usar el notebook que vimos en clase la referencia es el siguiente curso. 
[Machine Learning Fundamentals UC San Diego](https://www.edx.org/es/course/machine-learning-fundamentals-2)

In [ ]:
from sklearn.datasets import load_wine
import scipy.stats

alcohol_levels = load_wine()['data'][:, [0]].ravel()
alcalinity_levels = load_wine()['data'][:, [3]].ravel()

alcohol_levels.shape, alcalinity_levels.shape

((178,), (178,))

a) A partir de esas dos caracteristicas del vino, calcule el vector $\vec{\mu}$ y la matrix de covarianza $\Sigma$, 

$$
\vec{\mu} = \begin{bmatrix}
\mu_x\\
\mu_y
\end{bmatrix}, \quad \Sigma = \begin{bmatrix}
\text{Var}(x) & \text{Cov}(x, y)\\
\text{Cov}(y, x) & \text{Var}(y)
\end{bmatrix}
$$

In [4]:
## Tu codigo aqui
mu = np.zeros(2)
covar = np.zeros((2, 2))



print(mu, covar)

[0. 0.] [[0. 0.]
 [0. 0.]]


b) A partir del la matriz de covarianza dibuje los puntos y la gaussiana bivariada, con las curvas de nivel.

El resultado se tiene que ver como lo siguiente, 

![](https://raw.githubusercontent.com/diegour1/MetodosComputacionales1/main/DataFiles/image11.png)

In [ ]:
# tu codigo aqui

c) Diagonalice la matriz de covarianza, es decir obtenga los valores propios y vectores propios de la matriz de covarianza

In [5]:
# Tu codigo aquí
eig_vals = np.zeros(2)
eig_vecs = np.zeros((2, 2))



print(eig_vals, eig_vecs)

[0. 0.] [[0. 0.]
 [0. 0.]]


d) A partir de los vectores propios y valores propios de la matriz, y el vector $\vec{\mu}$.

*   Desplace los datos de forma que el promedio de los datos en juntas coordenadas sea cero.
*   Rote los datos de forma que la covarianza de los datos sea cero.
*   Gráfique la gaussiana de los nuevos datos rotados con sus curvas de nivel, a partir de la diagonzalización de la matrix de covarianza original (parte c). No volver a usar la función `np.cov()` de numpy.

Una rotación de puntos en dos dimensiones se puede ver como una matriz de la forma,

$$
\text{Rot}(\theta) = \begin{bmatrix}
\cos{\theta} & -\sin{\theta}\\
\sin{\theta} & \cos{\theta}
\end{bmatrix}
$$

La gráfica se debe ver como lo siguiente,

![](https://raw.githubusercontent.com/diegour1/MetodosComputacionales1/main/DataFiles/image12.png)

In [ ]:
# tu codigo aqui




Nota: Las coordenadas de los nuevos datos se llaman Componentes principales, lo que hicimos fue basicamente el método PCA, el cual permite reducir la dimensionalidad de los datos, ignorando las componentes menos importantes.

## Problema 3

Un método para medir el nivel de pH de una solución resulta en una medición con un valor que está distribuido de forma normal con un promedio igual al nivel de pH y una desviación estándar de 0.05. Un ingeniero ambiental afirma que las dos soluciones vienen de la misma fuente. Si esta hipótesis fuera correcta entonces el nivel de pH de las soluciones sería igual. Para probar la veracidad de esta afirmación se realizaron 10 mediciones independientes del nivel de pH de juntas soluciones, los datos son los siguientes.

![](https://raw.githubusercontent.com/diegour1/MetodosComputacionales1/main/DataFiles/image10.png)

1.   Calcule el $p$ value, guárdelo en la variable `p_value`.
2.   Qué puede concluir con un level of significance de $\alpha = 0.05$, guarde la conclusión en el string `conclusion`

Incluya el siguiente `print` al final:

```
print(f"p value = {p_value}, la conclusion es:  {conclusion}")
```





In [ ]:
# tu codigo aqui
p_value = 0
conclusion = ""



print(p_value)
print(f"p value = {p_value}, la conclusion es:  {conclusion}")

0
p value = 0, la conclusion es:  
